In [ ]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import qrcode
import matplotlib.image as mpimg
from skimage import io
from skimage import transform as tf
from random import uniform, randint
from Canny import *

import pylops
import os

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 15.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

## QR Code Creation

In [ ]:
FOLDER = 'test_dataset\\original'

if not os.path.isdir(FOLDER):
    os.mkdir(FOLDER)

for data in np.random.choice(np.arange(1000, 10000), size=10, replace=False):
    for version in [1, 2, 3, 4]:
        qr = qrcode.QRCode(
            version=version,
            error_correction=qrcode.constants.ERROR_CORRECT_M,
            box_size=10,
            border=4
        )
        qr.add_data(data)
        qr.make(fit=True)

        img = qr.make_image(fill_color='black', back_color='white')
        img = np.float32(np.asarray(img)) * 255
        img = np.dstack((img, img, img))

        cv2.imwrite(os.path.join(FOLDER, f"{data}-v{version}.png"), img)

In [ ]:
from os import listdir
from os.path import isfile, join
# images = [f for f in listdir('qr_dataset//original') if isfile(join('qr_dataset//original', f))]
images = [f for f in listdir('test_dataset//original') if isfile(join('test_dataset//original', f))]

## Smoothing

### Example

We select Gaussian Filter with size 25x25 and sigma value 7, because these images cannot be read by QR-code readers, such as OpenCV or built-in mobile phone readers. Any larger values result in too blurred images.

In [ ]:
img = io.imread('qr_dataset\\original\\' + images[0], as_gray=True)
smoothed_images = [img]
values = [1, 2, 5, 10]
fig_titles = ['Original image', 'Smoothed image', 'Smoothed image (2x)', 'Smoothed image (5x)', 'Smoothed image (10x)']
splots = 5

plt.subplot(1,splots,1)
plt.imshow(img)
plt.title(fig_titles[0])
plt.axis('off')

for i in range(4):
    kernel = gaussian_kernel(5*values[i], 1.4*values[i])
    smoothed = conv(img, kernel)

    plt.subplot(1,splots,i+2)
    plt.imshow(smoothed)
    plt.title(fig_titles[i+1])
    plt.axis('off')
    smoothed_images.append(smoothed)

plt.show()

### Adding to the dataset

We apply this blur to every image in the original folder.

In [ ]:
# kernel = gaussian_kernel(5*5, 1.4*5)
# for image in images:
#     image_name = 'qr_dataset//blurred//' + image
#     img = io.imread('qr_dataset\\original\\' + image, as_gray=True)
#     smoothed = conv(img, kernel)
#     io.imsave(image_name, smoothed)

In [ ]:
def uniform_disk_kernel(size, R):
    
    kernel = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            if(i**2+j**2<=R**2):
                kernel[i][j] = 1./(np.pi*R**2)
    return kernel

In [ ]:
for path in images:
    Nz=300
    Nx=300
    img = io.imread(path, as_gray=True)
    img = cv2.resize(img, dsize=(Nz, Nx), interpolation=cv2.INTER_CUBIC)

    size=10
    R=30

    h=uniform_disk_kernel(size, R)

    Cop = pylops.signalprocessing.Convolve2D(
      dims=(Nz, Nx), h=h, offset=(size // 2, size // 2), dtype="float32"
    )
    image_name = path.replace('original', 'blurred')
    imblur = Cop * img

    io.imsave(image_name, imblur)

## Skew

### Example

We select 4 different angles of skew. Any smaller values result in too skewed image, that would be almost impossible to read, while larger values do the same, but also block the finder patters.

In [ ]:
from skew import *

skew = Skew()
image = Image.open('qr_dataset/original/1077-v1.png')
image = np.asarray(image)
degrees = [70, 80, 95, 100]
for i in range(4):
    skewed_image_canvas = skew.canvasSize(image, degrees[i])
    newImage_30 = skew.shear(image, degrees[i], skewed_image_canvas)
    plt.imshow(newImage_30)
    plt.show()

### Adding to the Dataset

In [ ]:
from PIL import Image

for image in images:
    image_name = 'test_dataset//skewed//' + image
    img = Image.open('test_dataset//original//' + image)
    img = np.asarray(img)
    deg = randint(0, 3)
    skewed_image_canvas = skew.canvasSize(img, degrees[deg])
    modified = skew.shear(img, degrees[deg], skewed_image_canvas)
    io.imsave(image_name, modified)

### Visualization

In [ ]:
plt.rcParams.update({'font.size': 22})

img = io.imread('qr_dataset\\original\\' + images[0], as_gray=True)

plt.subplot(1, 4, 1)
plt.imshow(img)
plt.axis('off')
plt.title(f'Original Image')

img = io.imread('qr_dataset\\blurred\\' + images[0].split('.')[0] + '-blurred.png', as_gray=True)

plt.subplot(1, 4, 2)
plt.imshow(img)
plt.axis('off')
plt.title(f'Blurred Image')

img = io.imread('qr_dataset\\skewed\\' + images[0].split('.')[0] + '-skewed.png', as_gray=True)

plt.subplot(1, 4, 3)
plt.imshow(img)
plt.axis('off')
plt.title(f'Skewed Image')

img = io.imread('qr_dataset\\QR-DN1.0\\extracted One\\test\\1500.jpg', as_gray=True)

plt.subplot(1, 4, 4)
plt.imshow(img)
plt.axis('off')
plt.title(f'QR-DN1.0 Image')